In [1]:
import pandas as pd
import numpy as np

In [2]:
lpl_world_bo5_match_df = pd.read_csv("lpl_world_bo5_match_df.csv",index_col=0)
lpl_world_bo5_game_df = pd.read_csv("lpl_world_bo5_game_df.csv",index_col=0)

In [3]:
def convert_to_minutes(time_delta):
    time_delta = pd.to_timedelta(time_delta)
    return time_delta.total_seconds() / 60

## KDA Who is 积分king?

Tian不准确

In [4]:
lpl_world_bo5_game_df = lpl_world_bo5_game_df.drop(columns=["Defeat","Victory"])
lpl_world_bo5_game_df[["K", "D", "A"]] = lpl_world_bo5_game_df["Score"].str.split("/", expand=True)
lpl_world_bo5_game_df["K"] = pd.to_numeric(lpl_world_bo5_game_df["K"], errors="coerce")
lpl_world_bo5_game_df["D"] = pd.to_numeric(lpl_world_bo5_game_df["D"], errors="coerce")
lpl_world_bo5_game_df["A"] = pd.to_numeric(lpl_world_bo5_game_df["A"], errors="coerce")

lpl_world_bo5_game_df.head()

,Player Name,Date,Game,Tournament,Type,Champion,Result,Score,Duration,Player ID,K,D,A
0,1xn,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Ezreal,Victory,4/0/5,0 days 00:25:01,5177,4,0,5
1,1xn,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Miss Fortune,Victory,5/0/8,0 days 00:23:20,5177,5,0,8
2,1xn,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Smolder,Victory,2/1/9,0 days 00:26:22,5177,2,1,9
3,1xn,2024-08-11,FPX vs TT,LPL Summer Playoffs 2024,LPL,Varus,Defeat,0/2/7,0 days 00:30:32,5177,0,2,7
4,1xn,2024-08-11,FPX vs TT,LPL Summer Playoffs 2024,LPL,Miss Fortune,Victory,2/0/6,0 days 00:41:31,5177,2,0,6


In [5]:
lpl_world_bo5_game_stats = lpl_world_bo5_game_df.copy()
## 局均人头
## 局均死亡
## 局均KDA
## 局均助攻


# Calculate KDA, handling cases where 'D' is 0
lpl_world_bo5_game_stats["KDA"] = (
    lpl_world_bo5_game_stats["K"] + lpl_world_bo5_game_stats["A"]
) / np.where(lpl_world_bo5_game_stats["D"] != 0, lpl_world_bo5_game_stats["D"], 1)

### 谁是积分王？——谁的生涯击杀数最多？谁的BO5小局均击杀数最多？谁又是助攻王？谁又总是在黑屏

In [61]:
jifenking = lpl_world_bo5_game_stats.groupby("Player Name").agg(
    total_kills=("K","sum"),
    total_games=("K","count"),
    total_death=("D","sum"),
    total_assist=("A","sum")
).reset_index()
jifenking["kill_rank"] = jifenking["total_kills"].rank(ascending=False, method="min").astype(int)
jifenking["death_rank"] = jifenking["total_death"].rank(ascending=False, method="min").astype(int)
jifenking["assist_rank"] = jifenking["total_assist"].rank(ascending=False, method="min").astype(int)
column_mapping = {
    "Player Name": "选手名称",
    "total_kills": "总击杀",
    "total_death": "总死亡",
    "total_assist": "总助攻",
    "total_games": "总场次",
    "kill_rank": "击杀排名",
    "death_rank": "死亡排名",
    "assist_rank": "助攻排名"
}
new_column_order = ["选手名称", "总场次", "总击杀", "击杀排名","总死亡","死亡排名", "总助攻","助攻排名"]
jifenking = jifenking.rename(columns=column_mapping)[new_column_order]
jifenking.sort_values(by="总场次",ascending=False).head(20)

,选手名称,总场次,总击杀,击杀排名,总死亡,死亡排名,总助攻,助攻排名
89,xiaohu,271,892,3,644,2,1576,3
84,meiko,251,204,29,680,1,2163,1
32,JackeyLove,231,1028,1,628,3,1331,9
37,Knight,224,927,2,452,14,1380,8
54,Scout,218,777,4,465,12,1153,12
76,Yagao,200,596,8,503,7,1301,10
52,RooKie,199,754,6,489,9,1203,11
1,369,196,511,11,537,5,1056,13
62,Tian,196,499,12,536,6,1469,6
9,Bin,191,612,7,478,10,954,15


In [21]:
lpl_world_bo5_game_df[lpl_world_bo5_game_df["Player Name"] == "Tianzhen"]

,Player Name,Date,Game,Tournament,Type,Champion,Result,Score,Duration,Player ID,K,D,A
5706,Tianzhen,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Rengar,Defeat,0/5/0,0 days 00:25:01,5178,0,5,0
5707,Tianzhen,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Maokai,Defeat,0/4/2,0 days 00:23:20,5178,0,4,2
5708,Tianzhen,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Maokai,Defeat,0/5/2,0 days 00:26:22,5178,0,5,2


### 2. 以上的数据按小局来考虑呢？
那随之而来的另一个问题是，以上的数据假如是平均到每个小局呢？在每个BO5的小局里，积分王又是谁？

In [62]:
# 按选手分组并计算总数
jifenking_per = lpl_world_bo5_game_stats.groupby("Player Name").agg(
    total_kills=("K", "sum"),
    total_games=("K", "count"),
    total_death=("D", "sum"),
    total_assist=("A", "sum")
).reset_index()

# 计算每场平均值
jifenking_per["avg_kills"] = jifenking_per["total_kills"] / jifenking_per["total_games"]
jifenking_per["avg_death"] = jifenking_per["total_death"] / jifenking_per["total_games"]
jifenking_per["avg_assist"] = jifenking_per["total_assist"] / jifenking_per["total_games"]

# 计算排名
jifenking_per["kill_rank"] = jifenking_per["avg_kills"].rank(ascending=False, method="min").astype(int)
jifenking_per["death_rank"] = jifenking_per["avg_death"].rank(ascending=False, method="min").astype(int)  # 死亡越少排名越高
jifenking_per["assist_rank"] = jifenking_per["avg_assist"].rank(ascending=False, method="min").astype(int)

# 重命名列名为中文
jifenking_per = jifenking_per.rename(columns={
    "Player Name": "选手",
    "total_kills": "总击杀",
    "total_games": "总小局数",
    "total_death": "总死亡",
    "total_assist": "总助攻",
    "avg_kills": "场均击杀",
    "avg_death": "场均死亡",
    "avg_assist": "场均助攻",
    "kill_rank": "击杀排名",
    "death_rank": "死亡排名",
    "assist_rank": "助攻排名"
})

# 按排名列顺序重新排序
jifenking_per = jifenking_per[[
    "选手", "总小局数", "场均击杀", "击杀排名",
    "场均死亡", "死亡排名",
    "场均助攻", "助攻排名"
]]

# 查看结果
jifenking_per.sort_values(by="场均击杀",ascending=False).head(20)


,选手,总小局数,场均击杀,击杀排名,场均死亡,死亡排名,场均助攻,助攻排名
18,Doggo,24,4.666667,1,2.416667,57,4.916667,56
43,Lwx,107,4.560748,2,2.654206,38,5.308411,46
19,Elk,169,4.526627,3,2.301775,64,5.585799,40
29,Hope,58,4.482759,4,2.655172,36,4.982759,55
32,JackeyLove,231,4.450216,5,2.718615,31,5.761905,38
82,huanfeng,57,4.157895,6,1.912281,83,5.263158,50
37,Knight,224,4.138393,7,2.017857,78,6.160714,31
23,GALA,145,4.103448,8,1.813793,84,4.889655,58
42,Light,142,4.063380,9,1.753521,85,4.302817,67
66,Vicla,3,4.000000,10,3.000000,17,3.666667,78


### 小局最强kda

In [66]:
lpl_world_bo5_game_stats["Duration_min"] = lpl_world_bo5_game_stats["Duration"].apply(convert_to_minutes)
lpl_world_bo5_game_stats.sort_values(by="KDA", ascending=False).head(15).drop(columns=["K","D","A","Type","Player ID","Champion","Duration"])["Duration_min"].mean()

34.48555555555556

In [64]:
lpl_world_bo5_game_stats.head()
grouped_stats = lpl_world_bo5_game_stats.groupby(
    ["Player Name", "Date", "Tournament", "Game"]
).agg(
    total_kills=("K", "sum"),
    total_death=("D", "sum"),
    total_assist=("A", "sum")
).reset_index()
grouped_stats["bo5_kda"] = (grouped_stats["total_kills"] + grouped_stats["total_assist"]) / np.where(grouped_stats["total_death"]==0, 1, grouped_stats["total_death"])
grouped_stats.sort_values("bo5_kda",ascending=False).head(15)

,Player Name,Date,Tournament,Game,total_kills,total_death,total_assist,bo5_kda
1440,meiko,2024-05-07,MSI 2024,Team Liquid vs Top Esports,5,1,37,42.0
120,Beichuan,2024-08-05,LPL Summer Playoffs 2024,OMG vs TT,4,1,37,41.0
310,Elk,2023-04-03,LPL Spring Playoffs 2023,WBG vs BLG,28,1,12,40.0
340,Feather,2024-08-05,LPL Summer Playoffs 2024,OMG vs TT,2,0,38,40.0
690,Light,2021-09-04,LPL Regional Finals 2021,LNG Esports vs Rare Atom,27,1,12,39.0
256,Crisp,2023-11-02,Worlds Main Event 2023,NRG vs WBG,3,1,34,37.0
658,Knight,2024-10-26,Worlds Main Event 2024,WBG vs BLG,19,1,18,37.0
729,Lwx,2019-11-10,World Championship 2019,G2 Esports vs FPX,21,0,14,35.0
1089,Tarzan,2021-09-04,LPL Regional Finals 2021,LNG Esports vs Rare Atom,9,0,26,35.0
0,1xn,2024-08-05,LPL Summer Playoffs 2024,OMG vs TT,11,1,22,33.0


In [60]:
grouped_stats = lpl_world_bo5_game_stats.groupby("Player Name").agg(
    total_kills=("K", "sum"),
    total_death=("D", "sum"),
    total_assist=("A", "sum"),
    小局数=("A", "count")
).reset_index()
grouped_stats["career_kda"] = (grouped_stats["total_kills"] + grouped_stats["total_assist"]) / np.where(grouped_stats["total_death"]==0, 1, grouped_stats["total_death"])


column_mapping = {
    "Player Name": "选手名称",
    "total_kills": "总击杀",
    "total_death": "总死亡",
    "total_assist": "总助攻",
    "小局数": "小局数",
    "career_kda": "BO5生涯KDA"
}

# 替换列名为中文
grouped_stats = grouped_stats.rename(columns=column_mapping)

grouped_stats.sort_values("BO5生涯KDA",ascending=False).head(15)

,选手名称,总击杀,总死亡,总助攻,小局数,BO5生涯KDA
0,1xn,29,9,50,8,8.777778
53,Ruler,481,204,582,122,5.210784
37,Knight,927,452,1380,224,5.103982
23,GALA,595,263,709,145,4.958175
20,Feather,6,18,83,8,4.944444
82,huanfeng,237,109,300,57,4.926606
42,Light,577,249,611,142,4.771084
8,Beichuan,25,26,94,12,4.576923
19,Elk,765,389,944,169,4.393316
72,Xiaohao,25,35,127,16,4.342857


### (KILL + A) per min 
BLG WBG 一局杀疯了，但是没有

In [70]:
lpl_world_bo5_game_stats["ka_per_min"] = (lpl_world_bo5_game_stats["K"] + lpl_world_bo5_game_stats["A"])/lpl_world_bo5_game_stats["Duration_min"]
ka = lpl_world_bo5_game_stats.sort_values(by="ka_per_min", ascending=False).head(15).drop(columns=["K","D","A","Type","Player ID","Champion","Duration", "KDA"])
column_mapping = {
    "Player Name": "选手",
    "Date": "日期",
    "Game": "小局数",
    "Tournament": "比赛",
    "Result": "结果",
    "Score": "比分",
    "Duration_min": "时长(分钟)",
    "ka_per_min": "分均K+A"
}

ka = ka.rename(columns=column_mapping)
ka

,选手,日期,小局数,比赛,结果,比分,时长(分钟),分均K+A
6018,WeiWei,2023-08-07,Top Esports vs WBG,LPL Regional Finals 2023,Victory,4/0/22,24.466667,1.062670
4267,ON,2024-08-17,BLG vs WBG,LPL Summer Playoffs 2024,Victory,2/5/25,25.650000,1.052632
2430,JackeyLove,2021-04-10,RNG vs Top Esports,LPL Spring Playoffs 2021,Victory,4/6/21,25.316667,0.987492
1575,Elk,2024-05-18,BLG vs T1,MSI 2024,Victory,6/0/17,23.683333,0.971147
4275,ON,2024-08-30,BLG vs WBG,LPL Summer Playoffs 2024,Victory,1/3/23,25.083333,0.956811
5977,Wei,2024-08-30,BLG vs WBG,LPL Summer Playoffs 2024,Victory,8/1/16,25.083333,0.956811
2985,Knight,2021-04-06,Top Esports vs Suning,LPL Spring Playoffs 2021,Victory,6/0/12,19.050000,0.944882
4204,ON,2023-07-26,BLG vs Top Esports,LPL Summer Playoffs 2023,Victory,0/1/21,22.816667,0.920380
3155,Knight,2024-08-30,BLG vs WBG,LPL Summer Playoffs 2024,Victory,10/1/13,25.083333,0.916944
3698,Mark,2023-03-31,Top Esports vs Team WE,LPL Spring Playoffs 2023,Victory,0/2/21,23.500000,0.893617


In [80]:
ka = lpl_world_bo5_game_stats.groupby(["Player Name"]).agg(
    total_kills=("K", "sum"),
    total_assist=("A", "sum"),
    total_duration=("Duration_min", "sum"),
    total_games = ("K","count")
).reset_index()

ka["ka_per_min"] = (ka["total_kills"] + ka["total_assist"])/ ka["total_duration"]

column_mapping = {
    "Player Name": "选手",
    "Date": "日期",
    "Game": "小局数",
    "Tournament": "比赛",
    "Result": "结果",
    "Score": "比分",
    "Duration_min": "时长(分钟)",
    "ka_per_min": "分均K+A",
    "total_kills": "总击杀",
    "total_games": "总小局数",
    "total_death": "总死亡",
    "total_assist": "总助攻",
    "avg_kills": "场均击杀",
    "avg_death": "场均死亡",
    "avg_assist": "场均助攻",
    "kill_rank": "击杀排名",
    "death_rank": "死亡排名",
    "assist_rank": "助攻排名",
    "total_duration":"总时长"
}

ka = ka.rename(columns=column_mapping)
ka.sort_values(by="分均K+A",ascending=False).head(20)




,选手,总击杀,总助攻,总时长,总小局数,分均K+A
20,Feather,6,83,239.933333,8,0.370936
8,Beichuan,25,94,357.016667,12,0.333318
0,1xn,29,50,239.933333,8,0.329258
48,ON,130,1474,4898.216667,153,0.327466
71,XUN,337,903,3850.283333,120,0.322054
37,Knight,927,1380,7186.983333,224,0.320997
35,Kael,7,69,238.216667,8,0.319037
78,Zhuo,44,596,2017.883333,63,0.317164
16,Cryin,142,338,1516.583333,47,0.316501
19,Elk,765,944,5422.916667,169,0.315144


avg victory duration vs avg defeat duration

In [90]:
avg_duration = lpl_world_bo5_game_stats.groupby(["Player Name","Result"])["Duration_min"].mean().reset_index()

In [113]:
players = win_rate[win_rate["total_games"] >= 100]["Player Name"].to_list()

In [116]:
win_rate = lpl_world_bo5_game_stats.groupby("Player Name").agg(
    total_games=("Result", "count"),
    victories=("Result", lambda x: (x == "Victory").sum())
).reset_index()

# Add win rate column
win_rate["win_rate"] = win_rate["victories"] / win_rate["total_games"]
df = avg_duration.merge(win_rate[["Player Name","win_rate"]], on="Player Name")
df = df[df["Player Name"].isin(players)]

reshaped_data = df.pivot(index="Player Name", columns="Result", values="Duration_min").reset_index()

# Merge with win rate for coloring
reshaped_data = reshaped_data.merge(df[["Player Name", "win_rate"]].drop_duplicates(), on="Player Name")

# Rename columns for clarity
reshaped_data = reshaped_data.rename(columns={"Defeat": "avg_duration_defeat", "Victory": "avg_duration_victory"})


In [121]:
import plotly.graph_objs as go
import numpy as np

# Example DataFrame reshaped_data
offset_increment = 0.3
offsets = np.linspace(-offset_increment, offset_increment, len(reshaped_data))

fig = go.Figure()

# Scatter plot with text and offset
fig.add_trace(go.Scatter(
    x=reshaped_data["avg_duration_defeat"] + offsets,
    y=reshaped_data["avg_duration_victory"] + offsets,
    mode='markers+text',
    text=reshaped_data["Player Name"],
    textposition='top center',
    marker=dict(
        size=12,
        color=reshaped_data["win_rate"],
        colorscale='Plasma_r',
        cmin=reshaped_data["win_rate"].min(),
        cmax=reshaped_data["win_rate"].max(),
        showscale=True,
        colorbar=dict(
            title="Win Rate",
            titleside='right',
            titlefont=dict(size=14),
            tickvals=np.linspace(reshaped_data["win_rate"].min(), reshaped_data["win_rate"].max(), 5),
            tickfont=dict(size=12),
            lenmode='fraction',
            len=0.5
        )
    ),
    showlegend=False
))

# Add diagonal reference line (optional)
min_range = min(reshaped_data["avg_duration_defeat"].min(), reshaped_data["avg_duration_victory"].min()) - 1
max_range = max(reshaped_data["avg_duration_defeat"].max(), reshaped_data["avg_duration_victory"].max()) + 1

fig.add_trace(go.Scatter(
    x=[min_range, max_range],
    y=[min_range, max_range],
    mode='lines',
    line=dict(color='red', dash='dash'),
))

# Update layout
fig.update_layout(
    title='Avg Duration (Defeat) vs Avg Duration (Victory) with Win Rate Coloring',
    xaxis=dict(title='Avg Duration for Losing (min)', range=[min_range, max_range]),
    yaxis=dict(title='Avg Duration for Winning (min)', range=[min_range, max_range]),
    template='plotly_white',
    height=600,
    width=800,
)

fig.show()


In [ ]:
lpl_world_bo5_game_stats.groupby("Player Name")

In [125]:
# bo5_kda = 
lpl_world_bo5_game_stats.head(1)

,Player Name,Date,Game,Tournament,Type,Champion,Result,Score,Duration,Player ID,K,D,A,KDA,Duration_min,ka_per_min
0,1xn,2024-08-05,OMG vs TT,LPL Summer Playoffs 2024,LPL,Ezreal,Victory,4/0/5,0 days 00:25:01,5177,4,0,5,9.0,25.016667,0.35976


### 使用英雄最多的选手

In [152]:
player_stats = lpl_world_bo5_game_stats.groupby("Player Name").agg(
    total_games=("Champion", "count"),         # Total games played
    unique_champions=("Champion", "nunique")  # Unique champions used
).reset_index()

player_stats = player_stats.rename(columns={
    "Player Name": "选手",
    "total_games": "小局数",
    "unique_champions": "BO5英雄数"
})
player_stats.sort_values(by="BO5英雄数",ascending=False).head(20)

,选手,小局数,BO5英雄数
89,xiaohu,271,47
54,Scout,218,38
37,Knight,224,37
52,RooKie,199,37
1,369,196,35
21,Flandre,140,34
84,meiko,251,34
36,Kanavi,146,32
62,Tian,196,31
76,Yagao,200,31


In [185]:
lpl_world_bo5_game_stats[lpl_world_bo5_game_stats["Player Name"]=="xiaohu"].Champion.unique()

array(['Lissandra', 'LeBlanc', 'Azir', 'Ryze', 'Varus', 'Vladimir',
       'Syndra', 'Karma', 'Malzahar', 'Taliyah', 'Aurelion Sol', 'Viktor',
       'Jayce', 'Ekko', 'Galio', 'Corki', 'Lucian', 'Cassiopeia',
       'Orianna', 'Swain', 'Sion', 'Morgana', 'Irelia', 'Zoe', 'Rumble',
       'Neeko', 'Ornn', 'Gnar', 'Sylas', 'Renekton', 'Gragas', 'Lee Sin',
       'Dr. Mundo', 'Viego', 'Kennen', 'Wukong', 'Gwen', 'Ahri', 'Vex',
       'Twisted Fate', 'Akali', 'Annie', 'Tristana', 'Zeri', 'Yone',
       'Smolder', 'Aurora'], dtype=object)

### 选手们的BO5本命英雄？——哪个英雄使用的最多？最被某个选手爱的英雄

In [182]:
# Assuming `lpl_world_bo5_game_stats` is the dataset
# Calculate total games and wins per champion per player
player_champion_stats = lpl_world_bo5_game_stats.groupby(["Player Name", "Champion"]).agg(
    champion_games=("Result", "count"),  # Total games played with this champion
    wins=("Result", lambda x: (x == "Victory").sum())  # Number of wins with this champion
).reset_index()

# Calculate win rate
player_champion_stats["win_rate"] = (player_champion_stats["wins"] / player_champion_stats["champion_games"]) * 100

# Calculate total games per player
total_games_per_player = lpl_world_bo5_game_stats.groupby("Player Name").size().reset_index(name="total_games")
player_champion_stats = player_champion_stats.merge(total_games_per_player, on="Player Name")

# Calculate pick rate for each champion
player_champion_stats["pick_rate"] = (player_champion_stats["champion_games"] / player_champion_stats["total_games"]) * 100

# Display the complete summary table
player_champion_stats.head(3)


,Player Name,Champion,champion_games,wins,win_rate,total_games,pick_rate
0,1xn,Ezreal,4,2,50.0,8,50.0
1,1xn,Miss Fortune,2,2,100.0,8,25.0
2,1xn,Smolder,1,1,100.0,8,12.5


In [199]:
lpl_world_bo5_game_stats["Champion"].value_counts().reset_index().head(15)


,Champion,count
0,Rakan,161
1,Aphelios,157
2,Nautilus,155
3,Leona,149
4,Zeri,145
5,Renekton,141
6,Rell,134
7,Varus,127
8,Azir,126
9,Jayce,126


In [205]:
high_pick_rate = player_champion_stats[player_champion_stats["total_games"]>=50].sort_values(by="pick_rate", ascending=False).head(20)
high_pick_rate = high_pick_rate.drop(columns=["wins","total_games"])
high_pick_rate.columns = [["姓名","英雄","使用次数","使用胜率","BO5选择率"]]
high_pick_rate

,姓名,英雄,使用次数,使用胜率,BO5选择率
1161,Zika,KSante,14,42.857143,25.925926
1164,Zika,Renekton,14,28.571429,25.925926
1256,shanji,KSante,17,41.176471,25.373134
1150,Zhuo,Rell,15,53.333333,23.809524
414,Hope,Aphelios,13,53.846154,22.413793
655,Mark,Nautilus,22,45.454545,22.222222
368,GALA,Kaisa,31,54.838710,21.379310
1184,huanfeng,Jhin,12,66.666667,21.052632
768,Ppgod,Rakan,13,38.461538,20.634921
993,Wayward,Gnar,15,46.666667,20.270270


In [214]:
top_players = player_stats[player_stats["小局数"] >= 100]["选手"].to_list()

In [217]:
highest_pick_rate = player_champion_stats.loc[
    player_champion_stats.groupby("Player Name")["pick_rate"].idxmax()
]


In [224]:
player_data = player_champion_stats[player_champion_stats["Player Name"] == "369"]
player_data[player_data["champion_games"] >= 3].sort_values(by="win_rate", ascending=False).iloc[0]

Player Name            369
Champion              Gnar
champion_games           3
wins                     3
win_rate             100.0
total_games            196
pick_rate         1.530612
Name: 11, dtype: object

In [244]:
from IPython.display import HTML, display

# Preserve the original paths by creating a copy for rendering
render_df = result_df.copy()

# Function to format percentages
def format_percentage(value):
    return f"{value:.2f}%"

# Apply percentage formatting to relevant columns
render_df["Most Used Win Rate"] = render_df["Most Used Win Rate"].apply(format_percentage)
render_df["Highest Win Rate"] = render_df["Highest Win Rate"].apply(format_percentage)
render_df["Lowest Win Rate"] = render_df["Lowest Win Rate"].apply(format_percentage)

# Function to convert image path to HTML <img> tag
def image_tag(path):
    return f'<img src="{path}" width="25">'

# Apply the function to image columns for rendering only
for col in ["Most Used Champ (Image)", "Highest Win Rate Champ (Image)", "Lowest Win Rate Champ (Image)"]:
    render_df[col] = render_df[col].apply(image_tag)

# Translate column names to Chinese
render_df = render_df.rename(columns={
    "Player Name": "选手",
    "Most Used Champ (Image)": "最常用",
    "Most Used Win Rate": "胜率",
    "Highest Win Rate Champ (Image)": "最高胜率",
    "Highest Win Rate": "胜率",
    "Lowest Win Rate Champ (Image)": "最低胜率",
    "Lowest Win Rate": "胜率"
})

# Render the DataFrame as HTML with images
html_table = render_df.to_html(escape=False, index=False)
display(HTML(html_table))


选手,最常用,胜率,最高胜率,胜率,最低胜率,胜率
369,,50.00%,,100.00%,,33.33%
Ale,,70.59%,,75.00%,,0.00%
Bin,,65.71%,,100.00%,,20.00%
Breathe,,57.14%,,100.00%,,0.00%
Crisp,,36.36%,,100.00%,,33.33%
Elk,,45.83%,,85.71%,,0.00%
Flandre,,55.56%,,80.00%,,0.00%
GALA,,54.84%,,72.73%,,0.00%
JackeyLove,,61.29%,,100.00%,,14.29%
Jiejie,,43.75%,,100.00%,,0.00%



考虑到被击杀的问题——BO5小局最强KDA是谁的哪场比赛？BO5大局最强KDA?生涯KDA ？ （考虑到是除法，所以大局KDA直接用小局KDA的平均数应该会存在不准确的情况，所以会重新计算，结果也会不一样）
时长对于KDA也有影响——哪一场比赛的谁的分均（K+A）最高？谁的BO5生涯 分均（K+A）最高 ？（作为辅助玩家，私心把助攻放了上去kkk）
图：BO5小局胜场平均时长vs败场平均时长（对于上一个帖平均时长的补充）
选手们的BO5本命英雄？——哪个英雄使用的最多？
选手们哪个英雄胜率最高？胜率最低？
英雄也有自己的KDA！——英雄们自己的生涯胜率KDA
倒反天罡！英雄们的“本命选手”是谁？——每个英雄使用胜率最高的选手
每个赛季最多出现的英雄和他们的胜率？（有些赛季可能S14还在活动的选手比较少，所以可能不会每个赛季都统计到）

### 每位选手BO5本命英雄？胜率最高的英雄？胜率最低的英雄？
### 每个英雄的选手熟练榜单~
### 英雄使用排名（per年份）/胜率排名 --- 英雄kda
### 
### Machine Learning Model and selection